In [1]:
import os
import pandas as pd
import glob
import pandas as pd
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
import oracledb
# from const import QUERY_CUSTOMERINFO,CONNECT_ORACLE
import sys
sys.path.append(os.path.abspath(''))
from dotenv import load_dotenv
load_dotenv() 


True

In [3]:
conn_oracle = jaydebeapi.connect(
            os.getenv("ORACLE_JCLASSNAME"),
            os.getenv("ORACLE_URL"),
            [os.getenv("ORACLE_USER"), os.getenv("ORACLE_PASSWORD")],
            os.getenv("ORACLE_JARS")
        )

In [4]:
sms_type = pd.read_sql("SELECT * FROM SUPAT.REF_SMS_WORDING", conn_oracle)

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_18892\3546113724.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sms_type = pd.read_sql("SELECT * FROM SUPAT.REF_SMS_WORDING", conn_oracle)


In [5]:
sms_type

,ID,SMS_TYPE,SMS_WORDING
0,1,C4C,สินเชื่อรถยนต์แคชจอยของคุณลงท้าย {4digit} ครบก...
1,2,Unsave,สินเชื่อลงท้าย {4digit} ของท่านมียอดค้างชำระ โ...
2,3,In-House Unsave,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
3,4,SF+ Unsave,สินเชื่อของท่านค้างชำระ {totalunpaidbalance} บ...
4,5,SF+ non assign,สินเชื่อ SF+ บัญชีลงท้าย {4digit} เกินกำหนดชำร...
5,6,Delayed Calling,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
6,7,Offer TDR (old),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
7,8,TDR (GPD/Normal),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
8,9,TDR (SPD only),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
9,10,Offer Settlement,KBJ Capital ขอเสนอส่วนลดพิเศษสูงสุด 30% สำหรับ...


In [9]:
gg = pd.read_sql( 'SELECT * FROM SUPAT."customer_current"', conn_oracle)

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_18892\2561880677.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gg = pd.read_sql( 'SELECT * FROM SUPAT."customer_current"', conn_oracle)


In [10]:
gg

,AS_OF_DATE,CONTRACT_NO_VAL,NATIONAL_ID,MONTHLY_INST_AMT,FIRST_DUE_DATE,MOBILE_PHONE_NO_VAL
0,2025-04-22 00:00:00,9000221092000027,3130300370958,1080.00,2021-10-01 00:00:00,None
1,2025-04-22 00:00:00,1000158100056298,1103701776145,NaN,2015-10-26 00:00:00,None
2,2025-04-22 00:00:00,7221270100062650,3361000624082,782.21,2023-02-01 00:00:00,None
3,2025-04-22 00:00:00,7230970101195164,3210500714927,0.00,2023-10-01 00:00:00,None
4,2025-04-22 00:00:00,1000159121091646,1340300089339,NaN,2017-02-01 00:00:00,None
...,...,...,...,...,...,...
1616568,2025-04-22 00:00:00,8002060051056071,1100200675572,NaN,None,None
1616569,2025-04-22 00:00:00,8002060101036019,1470100170557,NaN,2017-12-01 00:00:00,None
1616570,2025-04-22 00:00:00,8002060121092771,1249900132765,NaN,None,None
1616571,2025-04-22 00:00:00,9000220062000443,1101800667296,NaN,2020-07-01 00:00:00,None


In [2]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True) 
    df.columns = df.columns.str.replace(r'loan_no', 'contract_no', regex=True)
    df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  
    return df


In [3]:
input_folder = "../checker/input/assign_input/"

In [4]:
files = glob.glob(os.path.join(input_folder, "*.xlsx")) + glob.glob(os.path.join(input_folder, "*.csv"))
print('files',files)

files ['../checker/input/assign_input\\Template_SMS_SF_X_DAY_Unsave_Collector_Covid_Base.xlsx']


In [5]:
if files:
    file_path = files[0]
    if file_path.endswith(".xlsx"):
        assign = pd.read_excel(file_path,sheet_name='Data', dtype={'MobileNo': str})
        print('assignassignassign',assign)
    elif file_path.endswith(".csv"):
        assign = pd.read_csv(file_path,sheet_name='Data', dtype={'MobileNo': str})   
else:
    print("No file found in the folder.")

assignassignassign       No          Loan No.  Customer ID No Customer Name/Surname(Thai)  \
0      1  7250270104814187   2320900048950                สุพิน บุญไทย   
1      2  7240770103411425   1520100123761            กวินตา สุราฤทธิ์   
2      3  7250170104669107   1319900790923                นราดร จีนกูล   
3      4  7241070104062890   1549900139899             จอมขวัญ พละบุรี   
4      5  7240870103435505   1469900500312         จักรพันธ์ ยุบลพันธ์   
..   ...               ...             ...                         ...   
315  316  7240270102091473   1100701547011              จริยา จันทวิลา   
316  317  7240770103292127   1530300063192            ปิยธิดา ลิตตะสาร   
317  318  7240770103301835   3849800093475                นิตยา ชูช่วง   
318  319  7241070104009274   1101800510265              สิริพร ศรีสกุล   
319  320  7241270104426728   3930500214413                 ฉลวย ศรีทอง   

     Product code Overdue Cnt(Morning)  Mobile No.  Status  Result_TD  \
0    A701:SF Loan  

In [7]:
conn = jaydebeapi.connect(
        os.getenv("ORACLE_JCLASSNAME"),
         os.getenv("ORACLE_URL"),
         [os.getenv("ORACLE_USER"), os.getenv("ORACLE_PASSWORD")],
         os.getenv("ORACLE_JARS"),
        )
cur = conn.cursor() 	


In [2]:
def read_file(pathfile,filename):
    if filename:
        if filename.endswith(".xlsx"):
            file = pd.read_excel(f'{pathfile}{filename}')
        elif filename.endswith(".csv"):
            encodings = ["utf-8", "ISO-8859-1", "latin1", "utf-16"]
            for enc in encodings:
                try:
                    file = pd.read_csv(f'{pathfile}{filename}')
                    # print(f"Successfully read {filename} using {enc} encoding.")
                    return file
                except UnicodeDecodeError:
                    print(f"Encoding error with {enc}, trying another...")
            # file = pd.read_csv(f'{pathfile}{filename}', encoding='ISO-8859-1')
        return file  
    # print(assign.head())  # Display first few rows
    else:
        print("No file found in the folder.")

In [5]:
file_current = glob.glob("./input/acc/*") 
file_current = file_current[0].split('\\')[1]
ar_file = read_file('./input/acc/',file_current)

In [6]:
ar_file

,BaseDate,Loan No,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OverdueCrUsage,OverdueOutputTax,OverdueCollectionFee,OverdueVAT,OverdueStampTaxAmt,OverdueNCBTaxAmt,OverdueETCFeeAmt,LawCost,LastPaymentDt,LastPaymentAmt
0,20250401,9010221122000949,A501:TL,2024-11-01,151,5:OD4,151,5:OD4,74164.44,33528.80,...,792.37,0.0,467.3,32.7,0,0,0.0,0.0,2025-03-31,3067.00
1,20250401,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,11088.74,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2024-09-27,20971.92
2,20250401,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-01-31,1300.00
3,20250401,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-12-27,550.00
4,20250401,9010221122000906,A501:TL,2022-11-30,0,98:Write-Off,0,98:Write-Off,35235.40,31005.17,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2022-09-30,2100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398867,20250401,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-01-31,1580.00
1398868,20250401,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-31,1260.00
1398869,20250401,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-16,4000.00
1398870,20250401,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-08-01,2600.00


In [7]:
ar_file.columns = ar_file.columns.str.lower()

In [9]:
ar_file.columns

Index(['basedate', 'loan no', 'product code', 'overdue start date',
       'overduedays_morning', 'overduecnt_morning', 'overduedays',
       'overduecnt', 'loan amt', 'loan balance', 'outstanding balance',
       'overduesumamt', 'overdueprin', 'overdueint', 'overduecrusage',
       'overdueoutputtax', 'overduecollectionfee', 'overduevat',
       'overduestamptaxamt', 'overduencbtaxamt', 'overdueetcfeeamt', 'lawcost',
       'lastpaymentdt', 'lastpaymentamt'],
      dtype='object')

In [10]:
ar_file[['loan no','overduesumamt']]

,loan no,overduesumamt
0,9010221122000949,16750.0
1,9010221122000931,0.0
2,9010221122000922,0.0
3,9010221122000914,0.0
4,9010221122000906,0.0
...,...,...
1398867,1000158090014831,100.0
1398868,1000158090014814,30.0
1398869,1000158090008580,100.0
1398870,1000158090005394,100.0
